In [1]:
!pip install spacy pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.8 MB/s eta 0:00:00


In [2]:
!python -m spacy download en_core_web_sm

2023-12-01 17:02:38.374644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 17:02:38.374721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 17:02:38.374779: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 17:02:38.388239: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 17:02:39.745846: W tensorflow/compiler/

Mounted at /content/drive


In [5]:
!pip install PyMuPDF
import fitz  # PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 43.6 MB/s eta 0:00:00


In [8]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text("text")
    return text

pdf_path = '/content/sample_data/calcrim_2023_edition.pdf'
pdf_text = extract_text_from_pdf(pdf_path)

In [15]:
print(pdf_text[:500])

Judicial Council of California
Criminal Jury Instructions
CALCRIM
2023
1
Series 100–1800
Judicial Council of California
Advisory Committee on Criminal Jury Instructions
Hon. Jeffrey S. Ross, Chair
LexisNexis Matthew Bender
Official Publisher
This version provided by LexisNexis® Matthew Bender®, Official Publisher, 800-533-1637,
store.lexisnexis.com, for public and internal court use
QUESTIONS ABOUT THIS PUBLICATION?
For questions about the Editorial Content appearing in these volumes or reprint 


In [10]:
import string

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Add more preprocessing steps as needed
    return text

preprocessed_text = preprocess_text(pdf_text)

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [13]:
def process_large_text(text):
    # Split the text into batches (adjust the batch size as needed)
    batch_size = 100000
    batches = [text[i:i + batch_size] for i in range(0, len(text), batch_size)]

    # Process each batch
    results = []
    for batch in batches:
        doc = nlp(batch)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        results.extend(entities)

    return results

In [14]:
def process_large_text_streaming(text):
    # Create a Doc object with streaming processing
    doc = nlp.make_doc(text)

    # Process the Doc in chunks
    results = []
    for chunk in doc.noun_chunks:
        # Process each chunk
        entities = [(ent.text, ent.label_) for ent in nlp(chunk.text).ents]
        results.extend(entities)

    return results

In [17]:
!pip install python-docx
import spacy
from docx import Document

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.6 MB/s eta 0:00:00


In [18]:
# Load English NLP model
nlp = spacy.load("en_core_web_sm")

In [21]:

import fitz  # PyMuPDF


# Example paths to PDF files
pdf1_path = '/content/sample_data/Incident_Report.pdf'
pdf2_path = '/content/sample_data/calcrim_2023_edition.pdf'

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        with fitz.open(pdf_path) as pdf_document:
            text = ""
            for page_number in range(pdf_document.page_count):
                page = pdf_document[page_number]
                text += page.get_text()
            return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Load the entire content of both PDF files
pdf1_text = extract_text_from_pdf(pdf1_path)
pdf2_text = extract_text_from_pdf(pdf2_path)

# Set the maximum number of characters or lines to print
max_characters_limit = 500  # Set your desired character limit
max_lines_limit = 10        # Set your desired line limit

# Print a limited portion of text from each PDF file
print("PDF 1 Text:")
print(pdf1_text[:max_characters_limit])
# or, if you want to limit by lines:
# print("\n".join(pdf1_text.split('\n')[:max_lines_limit]))

print("\nPDF 2 Text:")
print(pdf2_text[:max_characters_limit])
# or, if you want to limit by lines:
# print("\n".join(pdf2_text.split('\n')[:max_lines_limit]))

PDF 1 Text:
Report Number: [123] 
  Date of Report: [2023-01-15] 
  Reporting Officer: [Officer Smith, Badge Number 456] Assignment: [Investigation] 
      I. INCIDENT DETAILS   
1.   Date and Time of Incident: [2023-01-10, 14:30] 
2.   Location of Incident: [123 Main Street, City ville] 
3.   Nature of Incident: [Robbery] 
4.   Injuries Observed: [None] 
5.   Initial Observations: [Unknown suspect entered the store] 
 
      II. VICTIM INFORMATION   
1.   Name: [John Doe] 
2.   Address: [456 Oak Avenue, To

PDF 2 Text:
Judicial Council of California
Criminal Jury Instructions
CALCRIM
2023
1
Series 100–1800
Judicial Council of California
Advisory Committee on Criminal Jury Instructions
Hon. Jeffrey S. Ross, Chair
LexisNexis Matthew Bender
Official Publisher
This version provided by LexisNexis® Matthew Bender®, Official Publisher, 800-533-1637,
store.lexisnexis.com, for public and internal court use
QUESTIONS ABOUT THIS PUBLICATION?
For questions about the Editorial Content appearing in

In [22]:

# Define specific entity categories for NER
entity_categories = ["DATE", "TIME", "GPE", "PERSON", "ORG", "CRIME"]

# Extract key entities from PDF 1 based on headings and specific categories
criteria_entities = []
for ent in nlp(pdf1_text).ents:
    if ent.label_ in entity_categories:
        criteria_entities.append((ent.text, ent.label_))

In [28]:
import spacy

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Set a higher max_length limit (adjust as needed)
nlp.max_length = 2000000  # Set a value appropriate for your text length


# Chunk size (adjust as needed)
chunk_size = 100000



# Split the text into chunks
chunks = [pdf2_text[i:i + chunk_size] for i in range(0, len(pdf2_text), chunk_size)]

# Process each chunk separately and accumulate the results
doc2 = None
for chunk in chunks:
    doc_chunk = nlp(chunk)
    if doc2 is None:
        doc2 = doc_chunk
    else:
        doc2 += doc_chunk

# Search logic: Find entities in PDF 2 based on criteria from PDF 1
matched_entities = []


for criteria_entity, criteria_label in criteria_entities:
    for search_ent in doc2.ents:
        # Check if the criteria entity matches a search entity
        if criteria_entity.lower() in search_ent.text.lower():
            matched_entities.append((criteria_entity, criteria_label, search_ent.text, search_ent.label_))

# Display matched entities
for match in matched_entities:
    print(f"Matched: {match[0]} ({match[1]}) in PDF 1 with {match[2]} ({match[3]}) in PDF 2")


In [29]:
# Create a template for the output report
report_template = """
Report:

Matched Entities:
{matched_entities}

Conclusions and Hypotheses:
[Provide your conclusions and hypotheses based on the matched entities]

Penal Code of Crime:
[Provide the penal code of the crime that closely matches the search criteria]

... (add more sections as needed)
"""

# Populate the report template with actual information
final_report = report_template.format(
    matched_entities="\n".join([f"{match[0]} ({match[1]}) in PDF 1 with {match[2]} ({match[3]})" for match in matched_entities]),
    # Add more fields as needed
)

# Display or save the final report
print(final_report)

# Save the modified Word document
output_path = '/content/drive/MyDrive/Police_Report_Project/final_report.docx'
doc = Document()
doc.add_paragraph(final_report)
doc.save(output_path)



Report:

Matched Entities:


Conclusions and Hypotheses:
[Provide your conclusions and hypotheses based on the matched entities]

Penal Code of Crime:
[Provide the penal code of the crime that closely matches the search criteria]

... (add more sections as needed)



In [32]:
# Extract all entities from PDF 1
pdf1_entities = [(ent.text.lower(), ent.label_) for ent in nlp(pdf1_text).ents]


# Perform NER on PDF 2 (search area)
doc2 = nlp(pdf2_text)

# Debugging: Print extracted entities from PDF 1
print("Extracted Entities from PDF 1:")
for ent in pdf1_entities:
    print(f"{ent[0]} ({ent[1]})")

# Define specific entity categories for NER
entity_categories = set(label for _, label in pdf1_entities)

# Search logic: Find entities in PDF 2 based on criteria from PDF 1
matched_entities = []

for criteria_entity, criteria_label in pdf1_entities:
    for search_ent in doc2.ents:
        # Case-insensitive matching
        if criteria_entity.lower() in search_ent.text.lower():
            matched_entities.append((criteria_entity, criteria_label, search_ent.text, search_ent.label_))

# Debugging: Print matched entities for verification
print("Matched Entities:")
for match in matched_entities:
    print(f"Matched: {match[0]} ({match[1]}) in PDF 1 with {match[2]} ({match[3]}) in PDF 2")


Extracted Entities from PDF 1:
123 (CARDINAL)
2023-01-15 (DATE)
smith (PERSON)
456 (CARDINAL)
i. incident (ORG)
1 (CARDINAL)
time of incident (ORG)
2023-01-10 (DATE)
2 (CARDINAL)
location of incident (ORG)
123 (CARDINAL)
main street (FAC)
3 (CARDINAL)
4 (CARDINAL)
injuries observed (PERSON)
5 (CARDINAL)
1 (CARDINAL)
john doe] 
 (PERSON)
2 (CARDINAL)
456 (CARDINAL)
townsville (GPE)
3 (CARDINAL)
555-1234 (DATE)
john.doe@email.com (PERSON)
4 (CARDINAL)
victim (PERSON)
1 (CARDINAL)
jane smith (PERSON)
555 (CARDINAL)
1 (CARDINAL)
2 (CARDINAL)
25-30 (CARDINAL)
approximately 6 feet (QUANTITY)
3 (CARDINAL)
park street] 
 
      v. evidence (FAC)
1 (CARDINAL)
photographs (PERSON)
2 (CARDINAL)
3 (CARDINAL)
4 (CARDINAL)
1 (CARDINAL)
interviews conducted (FAC)
victim (PERSON)
2 (CARDINAL)
3 (CARDINAL)
additional investigative actions (ORG)
1 (CARDINAL)
2 (CARDINAL)
3 (CARDINAL)
johnson (PERSON)
officer's narrative of the incident (WORK_OF_ART)
any initial conclusions (ORG)
2023-01-15 (DATE)
Matche